# Miscellaneous investigations into the stochastic Lie algebra

## Poormans symbolic Pade approximation

Using the representation of rationals as pairs of arbitrary precision integers we can effectively symbolic compute the Pade approximation of Taylor series with rational coefficients.

We will use the Polynomials packages, along with the base rational numbers, and arbitrary precision integers.

In [2]:
using Polynomials
using Gadfly

In this first study we will formulate Pade approximation the Taylor series with coefficients given by inverse of the factorial of the arithmetic progression, with $a > 0$ and $b \ge 0$:
$$ 
\sum_{n=0}^\infty \frac{x^n}{\left(a\cdot n+b\right)!}
$$
In particular we are interested in $a=1$ and $b=2$

To compute the Pade coefficients we will use the following procedure:

1. Cast the series as rational arbitrary precision numbers
2. Then construct a polynomial
3. Finally find the rational Pade approximation

For ease of use we can turn this into a function to compute the coefficients of the Pade approximation of the inverse factorial arithmetic progression:

In [54]:
function padeprogression{T<:Integer}(n::T, m::T, a::T, b::T)
    if n < 1 || m < 1 || a < 1 || b < 0
        throw(AssertionError("Parameters out of range"))
    end
    l = n + m + 1
    u = Poly(ones(Rational{BigInt}, l))
    aB = convert(BigInt, a)
    dB = convert(BigInt, b - a)
    for k = 1:l
        dB += aB
        @inbounds u.a[k] /= factorial(dB)
    end
    Pade(u, n, m)
end

padeprogression (generic function with 1 method)

We can test the function

In [57]:
a=padeprogression(5, 5, 1, 0)

Polynomials.Pade{Rational{BigInt},Rational{BigInt}}(Poly(1//1 + 1//2x + 1//9x^2 + 1//72x^3 + 1//1008x^4 + 1//30240x^5),Poly(1//1 - 1//2x + 1//9x^2 - 1//72x^3 + 1//1008x^4 - 1//30240x^5),:x)

Compare this to the examples available online at [Wikipedia](https://en.wikipedia.org/wiki/Pad%C3%A9_approximant#Examples) and [Wolfram](http://mathworld.wolfram.com/PadeApproximant.html).

Exploring the numerator

In [31]:
a.p.a

6-element Array{Rational{BigInt},1}:
 1//1    
 1//2    
 1//9    
 1//72   
 1//1008 
 1//30240

Exploring the denominator

In [32]:
a.q.a

6-element Array{Rational{BigInt},1}:
  1//1    
 -1//2    
  1//9    
 -1//72   
  1//1008 
 -1//30240

Checking the type resolution, currently the final call to `Pade()` introduces an unavoidable type ambiguity

In [55]:
@code_warntype padeprogression(8, 8, 1, 2)

Variables:
  n::Int64
  m::Int64
  a::Int64
  b::Int64
  #s51::Bool
  #s50::Bool
  #s49::Bool
  #s48::Bool
  l::Int64
  u::Polynomials.Poly{Rational{BigInt}}
  aB::BigInt
  dB::BigInt
  #s52::Int64
  k::Int64
  ##xs#8191::Tuple{}
  ##dims#8192::Tuple{Int64}

Body:
  begin  # In[54], line 2:
      NewvarNode(symbol("#s50"))
      NewvarNode(symbol("#s49"))
      NewvarNode(symbol("#s48"))
      NewvarNode(:l)
      NewvarNode(:u)
      NewvarNode(:aB)
      NewvarNode(:dB)
      #s51 = (Base.slt_int)(n::Int64,1)::Bool
      unless #s51::Bool goto 0
      #s48 = #s51::Bool
      goto 3
      0: 
      #s50 = (Base.slt_int)(m::Int64,1)::Bool
      unless #s50::Bool goto 1
      #s48 = #s50::Bool
      goto 3
      1: 
      #s49 = (Base.slt_int)(a::Int64,1)::Bool
      unless #s49::Bool goto 2
      #s48 = #s49::Bool
      goto 3
      2: 
      #s48 = (Base.slt_int)(b::Int64,0)::Bool
      3: 
      GenSym(0) = #s48::Bool
      unless GenSym(0) goto 4 # In[54], line 3:
      (Main.throw)

For comparison here is the largest float, we have a little ways to go

In [40]:
[collect(62:64) [2^n - 1 for n = 62:64] [2^n for n = 62:64]]

3x3 Array{Int64,2}:
 62  4611686018427387903   4611686018427387904
 63  9223372036854775807  -9223372036854775808
 64                   -1                     0

In [58]:
c=padeprogression(12,13,1,2)

Polynomials.Pade{Rational{BigInt},Rational{BigInt}}(Poly(1//2 - 2//27x + 11//648x^2 - 11//8100x^3 + 1//7200x^4 - 1//144900x^5 + 1//2484000x^6 - 1//78246000x^7 + 1//2253484800x^8 - 1//117744580800x^9 + 1//6279710976000x^10 - 1//693908062848000x^11 + 1//116576554558464000x^12),Poly(1//1 - 13//27x + 1//9x^2 - 11//675x^3 + 11//6480x^4 - 11//82800x^5 + 1//124200x^6 - 1//2608200x^7 + 1//69552000x^8 - 1//2378678400x^9 + 1//107040528000x^10 - 1//6672192912000x^11 + 1//640530519552000x^12 - 1//124903451312640000x^13),:x)

In [59]:
[c.p.a[end].den c.q.a[end].den; 2^63 - 1 2^63 - 1 ; c.p.a[end].den - (2^63 - 1) c.q.a[end].den - (2^63 - 1)]

3x2 Array{BigInt,2}:
   116576554558464000    124903451312640000
  9223372036854775807   9223372036854775807
 -9106795482296311807  -9098468585542135807

## Logarithms of permutation matrices

Every permutation matrix is the product of cyclic permutation matrices; so we start be writing a function to construct the $k$ of $\left(n-1\right)!$ cyclic permutations of length $n$. Starting with the dense call:

In [60]:
function cyclicpermutation{T<:Number, S<:Integer}(::Type{T}, n::S, k::S)
    if n < 2 || k < 1
        throw(AssertionError("Parameters out of range"))
    end
    u = [1; nthperm(collect(2:n), BigInt(k))]
    A = zeros(T, n, n)
    a = convert(T, 1)
    for i = 1:n
        @inbounds A[u[i], u[mod(i, n) + 1]] = a
    end
    return A
end
cyclicpermutation{S<:Integer}(n::S, k::S) = cyclicpermutation(Int64, n, k)

cyclicpermutation (generic function with 2 methods)

A quick test generating a dense permutation matrix $A_{\pi\left(\cdot\right)}$

In [61]:
A = cyclicpermutation(5, 1)

5x5 Array{Int64,2}:
 0  1  0  0  0
 0  0  1  0  0
 0  0  0  1  0
 0  0  0  0  1
 1  0  0  0  0

In [63]:
det(A)

1.0

Checking the type resolution

In [67]:
@code_warntype cyclicpermutation(8, 100)

Variables:
  n::Int64
  k::Int64

Body:
  begin  # In[60], line 13:
      return (Main.cyclicpermutation)(Main.Int64,n::Int64,k::Int64)::Array{Int64,2}
  end::Array{Int64,2}


The sparse version:

In [64]:
function spcyclicpermutation{T<:Number, S<:Integer}(::Type{T}, n::S, k::S)
    if n < 2 || k < 1
        throw(AssertionError("Parameters out of range"))
    end
    u = [1; nthperm(collect(2:n), BigInt(k))]
    v = [u[2:end]; 1]
    w = ones(T, n)
    sparse(u, v, w, n, n)
end
spcyclicpermutation{S<:Integer}(n::S, k::S) = spcyclicpermutation(Int64, n, k)

spcyclicpermutation (generic function with 2 methods)

...and the sparse call

In [68]:
B = spcyclicpermutation(6, 120)

6x6 sparse matrix with 6 Int64 entries:
	[2, 1]  =  1
	[3, 2]  =  1
	[4, 3]  =  1
	[5, 4]  =  1
	[6, 5]  =  1
	[1, 6]  =  1

In [69]:
det(B)

-1.0

Checking the type resolution

In [70]:
@code_warntype spcyclicpermutation(7, 120)

Variables:
  n::Int64
  k::Int64

Body:
  begin  # In[64], line 10:
      return (Main.spcyclicpermutation)(Main.Int64,n::Int64,k::Int64)::SPARSEMATRIXCSC{TV,T}
  end::SPARSEMATRIXCSC{TV,T}


The permutation matrices $A_{\pi\left(\cdot\right)}$ are related to the canonical generators of the stochastic Lie algebra through $A_{\pi\left(\cdot\right)} = C_{\pi\left(\cdot\right)} + I$, where:
$$
    \begin{eqnarray}
        C_{\pi\left(\cdot\right)}
            & = & \sum_{i=1}^n C_{i \pi\left(i\right)}\\
            & = & \sum_{i=1}^n \hat{e}_i \otimes \hat{e}_{\pi\left(i\right)} - \hat{e}_i \otimes \hat{e}_i
    \end{eqnarray}
$$
For example:

In [44]:
full(B) - I

6x6 Array{Int64,2}:
 -1   0   0   0   0   1
  1  -1   0   0   0   0
  0   1  -1   0   0   0
  0   0   1  -1   0   0
  0   0   0   1  -1   0
  0   0   0   0   1  -1

Lets take a look at some logarithms:

In [45]:
C = cyclicpermutation(4, 1)

4x4 Array{Int64,2}:
 0  1  0  0
 0  0  1  0
 0  0  0  1
 1  0  0  0

In [46]:
round(logm(C), 5)

4x4 Array{Complex{Float64},2}:
     0.0+0.7854im   0.7854-0.7854im     -0.0+0.7854im  -0.7854-0.7854im
 -0.7854-0.7854im      0.0+0.7854im   0.7854-0.7854im      0.0+0.7854im
     0.0+0.7854im  -0.7854-0.7854im      0.0+0.7854im   0.7854-0.7854im
  0.7854-0.7854im     -0.0+0.7854im  -0.7854-0.7854im      0.0+0.7854im

By definition the $\ln A_{\pi\left(\cdot\right)}$ must belong to the same Lie subalgebra as $C_{\pi\left(\cdot\right)} = A_{\pi\left(\cdot\right)} - I$.

Furthermore the Lie subalgebra of $\ln A_{\pi\left(\cdot\right)}$ must contain the positive powers $C_{\pi\left(\cdot\right)}^n$, which are given by the binomial sums:
$$
    C_{\pi\left(\cdot\right)}^n = \sum_{k=0}^n \left(-1\right)^{n-k} \binom{n}{k} C_{\pi^n \left(\cdot\right)}
$$
Where $C_{\pi^0 \left(\cdot\right)} = 0$.

As well we trivially have that $\left[C_{\pi^n \left(\cdot\right)}, C_{\pi^m \left(\cdot\right)}\right] = 0$ because $A_{\pi\left(\cdot\right)}^n=A_{\pi^n \left(\cdot\right)}$

Finally if $\pi \left(\cdot\right)$ has period $p$, so that $\pi^p \left(i\right) = i$ then
$$
    C_{\pi^n \left(\cdot\right)} = C_{\pi^{n\bmod p} \left(\cdot\right)}
$$

From which we can deduce that:
$$
    A_{\pi\left(\cdot\right)} = \prod_{n=1}^{p-1} \exp\left(\alpha_n C_{\pi^n \left(\cdot\right)} \right)
$$

Critically the coefficients $\alpha_n$ are independent of $\pi\left(\cdot\right)$, and depend only on $p$.

When $p$ is even the magnitude of the real parts of $\alpha_p = -\alpha_{p-1}$ are the $p$ outer Pythagorean approximation of $\pi$

$$
\alpha_n = \left(-1\right)^{n+1} \frac{\pi}{p} \left(\frac{1}{\tan\left(n \frac{\pi}{p}\right)} +  \mathrm{i}\right)
$$

When $p$ is odd the magnitude of the real parts of $\alpha_p = -\alpha_{p-1}$ are the $p$ inner Pythagorean approximation of $\pi$

$$
\alpha_n = \left(-1\right)^{n+1} \frac{\pi}{p} \frac{1}{\sin\left(n \frac{\pi}{p}\right)}
$$

We start with a function to calculate coefficients

In [71]:
function pythagoreancoefficients{S<:Integer, T<:Real}(::Type{T}, n::S)
    if n < 2
        throw(AssertionError("Parameters out of range"))
    end
    u = Vector{Complex{T}}(n - 1)
    nT = convert(T, n)
    piT = convert(T, pi)
    oT = convert(T, 1)
    noT = convert(T, -1)
    zT = convert(T, 0)
    sT = noT
    aT = zT
    if n % 2 == 0
        for m = 1:(n - 1)
            sT *= noT
            aT += oT
            @inbounds u[m] = complex(sT * piT / (nT * tan(piT * (aT / nT))), sT * piT / nT)
        end
    else
        for m = 1:(n - 1)
            sT *= noT
            aT += oT
            @inbounds u[m] = complex(sT * piT / (nT * sin(piT * (aT / nT))), zT)
        end
    end
    return u
end
pythagoreancoefficients{S<:Integer}(n::S) = pythagoreancoefficients(Float64, n::S)

pythagoreancoefficients (generic function with 2 methods)

Another quick test of the new functions

In [83]:
pythagoreancoefficients(10)

9-element Array{Complex{Float64},1}:
    0.966883+0.314159im
   -0.432403-0.314159im
     0.22825+0.314159im
   -0.102077-0.314159im
 1.92367e-17+0.314159im
    0.102077-0.314159im
    -0.22825+0.314159im
    0.432403-0.314159im
   -0.966883+0.314159im

Checking type resolution

In [73]:
@code_warntype pythagoreancoefficients(23)

Variables:
  n::Int64

Body:
  begin  # In[71], line 28:
      return (Main.pythagoreancoefficients)(Main.Float64,n::Int64)::Array{Complex{Float64},1}
  end::Array{Complex{Float64},1}


We can implement this in a function that will always return a dense complex matrix:

In [74]:
function logcyclicpermutation{S<:Integer, T<:Real}(::Type{T}, n::S, k::S)
    if n < 2 || k < 1
        throw(AssertionError("Parameters out of range"))
    end
    A = cyclicpermutation(Complex{T}, n, k)
    B = zeros(A)
    C = eye(A)
    u = pythagoreancoefficients(T, n)
    for m = 1:(n - 1)
        C *= A
        @inbounds B += (C - I) * u[m]
    end
    return B
end
logcyclicpermutation{S<:Integer}(n::S, k::S) = logcyclicpermutation(Float64, n, k)

logcyclicpermutation (generic function with 2 methods)

Testing the function:

In [81]:
round(expm(logcyclicpermutation(8, 1)), 5)

8x8 Array{Complex{Float64},2}:
  0.0-0.0im   1.0+0.0im   0.0+0.0im  …  -0.0+0.0im  -0.0+0.0im  -0.0-0.0im
  0.0-0.0im   0.0+0.0im   1.0-0.0im      0.0-0.0im  -0.0+0.0im   0.0-0.0im
  0.0-0.0im  -0.0-0.0im   0.0-0.0im      0.0+0.0im   0.0+0.0im  -0.0+0.0im
  0.0+0.0im   0.0-0.0im  -0.0+0.0im     -0.0+0.0im   0.0+0.0im   0.0+0.0im
 -0.0+0.0im   0.0-0.0im   0.0+0.0im      1.0+0.0im   0.0+0.0im  -0.0-0.0im
  0.0+0.0im  -0.0-0.0im  -0.0+0.0im  …   0.0-0.0im   1.0+0.0im   0.0+0.0im
 -0.0-0.0im   0.0+0.0im   0.0+0.0im     -0.0+0.0im   0.0+0.0im   1.0+0.0im
  1.0-0.0im   0.0+0.0im  -0.0-0.0im      0.0-0.0im  -0.0-0.0im   0.0+0.0im

Checking the type resolution

In [77]:
@code_warntype logcyclicpermutation(9,15)

Variables:
  n::Int64
  k::Int64

Body:
  begin  # In[74], line 15:
      return (Main.logcyclicpermutation)(Main.Float64,n::Int64,k::Int64)::Array{Complex{Float64},2}
  end::Array{Complex{Float64},2}
